### Training and prediction functions

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.contrib.rnn import *
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU, Dropout, LSTM, Flatten, CuDNNLSTM
from keras.optimizers import adam


class RNN:
    def __init__(self, num_features: int):
        self.history = None
        self.num_features = num_features
        self.scaler = MinMaxScaler(feature_range=(0, self.num_features))

        self.model = Sequential()
        self.model.add(LSTM(64, input_shape=(self.num_features, 1)))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(1))
        self.model.compile(optimizer=adam(lr=0.005), loss="mae")

    def train(self, data, batch_size: int = 32, epochs: int = 20):
        feature_train, target_train = self.generate_feature_and_target(data)
        self.history = self.model.fit(feature_train, target_train, epochs=epochs, batch_size=batch_size)
        return self.history

    def forecast(self, feature: pd.core.frame.DataFrame):
        feature_test = feature.reshape(-1, self.num_features)
        scaled = self.scaler.transform(feature_test)

        feature_test = np.array(scaled[:, :self.num_features])
        feature_test = np.reshape(feature_test, (feature_test.shape[0], feature_test.shape[1], 1))

        forecasts = self.model.predict(feature_test)
        return forecasts[-1]

    def generate_feature_and_target(self, data: pd.core.frame.DataFrame):
        feature_train = self.scaler.fit_transform(data[:, :self.num_features])
        feature_train, target_train = np.array(feature_train[:, :self.num_features]), np.array(data[:, self.num_features])

        feature_train = np.reshape(feature_train, (feature_train.shape[0], feature_train.shape[1], 1))

        return feature_train, target_train

Using TensorFlow backend.
